In [1]:
cd ..

In [2]:
! ls data
! rm -rf data/weather_backup.parquet
! ls data

accident-montreal-documentation.pdf  road-network
accidents-montreal.lock		     road-network.lock
accidents-montreal.parquet	     road-network.parquet
accidents-montreal.zip		     weather


In [3]:
from accidents_montreal import fetch_accidents_montreal,\
                               extract_accidents_montreal_df
from road_network import fetch_road_network, extract_road_segments_df
from weather import get_weather
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import atan2, sqrt, row_number, cos, sin, radians,\
                                  col, rank, avg
from pyspark.sql import functions, types, Row

In [ ]:
def init_spark():
    return (SparkSession
            .builder
            .getOrCreate())


# init spark
spark = init_spark()
sc = spark.sparkContext

# retrieve datasets
fetch_accidents_montreal()
accidents_df = extract_accidents_montreal_df(spark)

Skip fetching montreal accidents dataset: already downloaded
Skip extraction of accidents montreal dataframe: already done, reading from file


In [ ]:
from pyspark.sql.types import *

non_num = ['TempFlag',
                    'DewPointTempFlag',
                    'RelHumFlag',
                    'WindDirFlag',
                    'WindSpdFlag',
                    'VisibilityFlag',
                    'StnPressFlag',
                    'HmdxFlag',
                    'WindChillFlag']

num = ['DewPointTemp°C',
                'RelHum%',
                'WindDir10sdeg',
                'WindSpdkm/h',
                'Visibilitykm',
                'StnPresskPa',
                'Hmdx',
                'WindChill'] + ['Temp°C']

schema = StructType(
    [StructField(c, StringType(), True) for c in non_num]
    + [StructField(c, FloatType(), True) for c in num]
    + [StructField('Weather', ArrayType(elementType=StringType()), True)])

In [ ]:
from pyspark.sql.functions import udf
import numpy as np
import os


def extract_date_val(i):
    return udf(lambda val : val.split('/')[i])

@udf
def extract_hour(val):
    return val.split('-')[0].split(':')[0]

test = accidents_df.select('DT_ACCDN','LOC_LAT', 'LOC_LONG', 'HEURE_ACCDN')  \
                            .withColumn("year", extract_date_val(0)(accidents_df.DT_ACCDN)) \
                            .withColumn("month", extract_date_val(1)(accidents_df.DT_ACCDN)) \
                            .withColumn("day", extract_date_val(2)(accidents_df.DT_ACCDN)) \
                            .withColumn("HEURE_ACCDN", extract_hour(accidents_df.HEURE_ACCDN)) \
                            .drop('DT_ACCDN') \
                            .replace('Non précisé', '00')

nb_elements_treated=0.0

def ai(row):
    new_row = get_weather(row.LOC_LAT, 
                                 row.LOC_LONG, 
                                 row.year, 
                                 row.month, 
                                 row.day, 
                                 row.HEURE_ACCDN)
    
    global nb_elements_treated
    nb_elements_treated +=1
    print('progress: ', nb_elements_treated*100/149886, '%')
    return new_row
    
backup_file = 'data/weather_backup.parquet'    
    
import time
t = time.time()
test2 = spark.createDataFrame(test \
        .sample(0.5) \
        .rdd \
        .map(lambda row: ai(row)), schema).write.parquet(backup_file)
t= time.time()-t
print('total processing time: ', t)


"""
#test for 1 row
row1 = spark.createDataFrame(test.limit(150).collect(), test.columns) \
    .rdd \
    .first()
rdd2 = sc.parallelize([row1]).map(lambda row: get_weather(int(row.LOC_LAT), 
                                 int(row.LOC_LONG), 
                                 int(row.year), 
                                 int(row.month), 
                                 int(row.day), 
                                 int(row.HEURE_ACCDN)))
print(rdd2.first())"""

In [ ]:
"""test = spark.read.parquet(backup_file)"""

#test3 = spark.createDataFrame(test2.collect(), list(a.asDict().keys()))
"""test4.write.parquet('data/test.parquet')"""

In [ ]:
"""test.limit(1).collect()"""

In [ ]:
""""df=extract_accidents_montreal_dataframe(spark)
test=df.take(5)
test = list(map(lambda e: (e.LOC_LAT, e.LOC_LONG), test))
t= test[0]
lat, long = t
year, month, day = (2006,5,2)
hour = 0
print(get_weather(lat, long, year, month, day, hour))""""

In [ ]:
"""import os
os.path.isfile('data/test.parquet')"""

In [ ]:
'(' not in "dsgarth"